In [1]:
from bs4 import BeautifulSoup
import re
from threading import Thread
import urllib
import pandas as pd
import urllib.request

In [2]:
opener = urllib.request.build_opener()
# IE 9 proved to be the most successful
opener.addheaders = [('User-agent', 'IE 9/Windows: Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)')]
urllib.request.install_opener(opener)

In [20]:
# Function that will do the scraping job from yelp
def scrape(ur):

    with urllib.request.urlopen(ur) as url:
        html = url.read()
    soup = BeautifulSoup(html,"lxml")
    retaurant_name = soup.find('h1')
    
    # create a dictionary business info for storing key business features 
    business_info = {}
    business_info['restaurant_name']= str(retaurant_name.text.strip().rstrip())
    
    retaurant_address = soup.find('span',itemprop="streetAddress")
    if retaurant_address != None:
        business_info['retaurant_address'] = str(retaurant_address.text.strip().rstrip())
    
    restaurant_zipcode = soup.find('span',itemprop="postalCode")
    if restaurant_zipcode != None:
        business_info['restaurant_zipcode'] = str(restaurant_zipcode.text.strip().rstrip())
    
    restaurant_reviewcount = soup.find('span',itemprop="reviewCount")
    if restaurant_reviewcount != None:
        business_info['restaurant_reviewcount'] = str(restaurant_reviewcount.text.strip().rstrip())
   
    if soup.find(itemprop="ratingValue") != None:
        business_info['restaurant_rating'] = soup.find(itemprop="ratingValue").get("content")

    
    hygiene_score = soup.find('dd',{'class':"nowrap health-score-description"})
    if hygiene_score != None:
        business_info['Hygiene_score'] = str(hygiene_score.text.strip().rstrip())
        
        
    price_range = soup.find('dd', {'class':"nowrap price-description"})
    if price_range != None:
        business_info['price_range'] = str(price_range.text.strip().rstrip())
   
    if soup.find('div',{'class':'short-def-list'}) != None:
        for i in soup.find('div',{'class':'short-def-list'}).findAll('dl'):
            key = i.find('dt').text.strip().rstrip()
            value = i.find('dd').text.strip().rstrip()
            business_info[str(key)]=str(value)
    
    #latitude = soup.find(property="place:location:latitude").get("content")
    #longitude = soup.find(property="place:location:longitude").get("content")
    
    return business_info

In [21]:
# Example for the scrape function
d = scrape('https://www.yelp.com/biz/blue-ribbon-sushi-new-york?osq=blue+ribbon')
print(d)

{'Parking': 'Street', 'restaurant_reviewcount': '1023', 'restaurant_name': 'Blue Ribbon Sushi', 'Attire': 'Casual', 'Wi-Fi': 'No', 'Take-out': 'Yes', 'Hygiene_score': 'A', 'price_range': '$31-60', 'Gender Neutral Restrooms': 'Yes', 'Good For': 'Dinner', 'Has TV': 'No', 'Takes Reservations': 'No', 'Wheelchair Accessible': 'No', 'restaurant_zipcode': '10012', 'Outdoor Seating': 'No', 'Alcohol': 'Full Bar', 'Delivery': 'No', 'restaurant_rating': '4.0', 'retaurant_address': '119 Sullivan St', 'Accepts Apple Pay': 'No', 'Bike Parking': 'Yes', 'Waiter Service': 'Yes', 'Good for Kids': 'No', 'Noise Level': 'Average', 'Good for Groups': 'Yes', 'Caters': 'No', 'Accepts Credit Cards': 'Yes'}


In [5]:
# List of yelp urls to scrape
def get_urls_from_search(term, location, num):
    
    term = term.replace(' ','+')
    location = location.replace(' ','+')
    query = 'https://www.yelp.com/search?find_desc='+term+'&find_loc='+location+'&start='+str(num*10)
    with urllib.request.urlopen(query) as url:
        contents = url.read()
    #contents = urllib.urlopen(query).read()
    soup = BeautifulSoup(contents, "html.parser")
    #print(soup)
    business_url = []
    for result in soup.findAll('a',{'class':'biz-name js-analytics-click'}):
        business_url.append("http://www.yelp.com" + result['href'])
    return business_url


In [7]:
searchLocations = ['East_Village', 'Upper_West_Side', 'Chelsea'] 

In [17]:
max_num = 1
urls_set = []
for i, loc in enumerate(searchLocations):
    # now run for loop with fix location and food type and append urls 
    # page ranking is based on relevance ranked by yelp
    for num in range(0,max_num):
        urls = get_urls_from_search('Japanese Food',loc, num)
        urls = urls[1:] # 0th link is irrelavant
        # len(urls)=0 if the starting page number exceed the maximum possible
        if (len(urls) ==0):
            break
        else:
            for i in range(0,len(urls)-1):
                urls_set.append(urls[i])
    #convert the urls from list to csv file for each location
    #pd_urls_set = pd.DataFrame(urls_set)
    #pd_urls_set.to_csv("urls_set_{0}.csv".format(loc))

In [18]:
print(len(urls_set))
urls_set

57


['http://www.yelp.com/biz/benemon-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/pado-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/raku-new-york-7?osq=Japanese+Food',
 'http://www.yelp.com/biz/izakaya-new-york-2?osq=Japanese+Food',
 'http://www.yelp.com/biz/jintan-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/suki-new-york-5?osq=Japanese+Food',
 'http://www.yelp.com/biz/kyo-ya-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/restaurant-ukiyo-new-york-2?osq=Japanese+Food',
 'http://www.yelp.com/biz/soba-ya-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/ichibantei-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/dokodemo-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/ise-restaurant-new-york-3?osq=Japanese+Food',
 'http://www.yelp.com/biz/secchu-yokota-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/yakitori-taisho-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz/kura-new-york?osq=Japanese+Food',
 'http://www.yelp.com/biz

In [22]:
info = {} # create a dictionary for srestaurant info
for u in urls_set:
    url_dict=scrape(u)
    for key,value in url_dict.items():
        info.setdefault(key,[]).append(value)

In [28]:
print(info['Parking'])

['Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Street', 'Private Lot', 'Street, Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Street', 'Private Lot', 'Private Lot', 'Private Lot', 'Street', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot', 'Private Lot']


In [29]:
header=info.keys()
info={}
for u in urls_set:
    url_dict=scrape(u)
    for i in header:
        if i in url_dict.keys():
            info.setdefault(i,[]).append(url_dict[i])
        else:
            info.setdefault(i,[]).append('NA')

In [30]:
final_df=pd.DataFrame(info)
final_df

,Accepts Apple Pay,Accepts Bitcoin,Accepts Credit Cards,Accepts Google Pay,Alcohol,Ambience,Attire,Best Nights,Bike Parking,Caters,...,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi,price_range,restaurant_name,restaurant_rating,restaurant_reviewcount,restaurant_zipcode,retaurant_address
0,No,No,Yes,No,Beer & Wine Only,"Trendy, Intimate",Casual,NA,No,No,...,No,Yes,No,No,$11-30,Benemon,4.5,432,10003,108 E 4th St
1,NA,NA,Yes,NA,NA,NA,Casual,NA,NA,NA,...,Yes,Yes,NA,NA,NA,Pado,5.0,14,10003,199 2nd Ave
2,No,NA,No,No,Beer & Wine Only,Casual,Casual,NA,Yes,No,...,No,Yes,No,No,$11-30,Raku,4.5,776,10003,342 E 6th St
3,No,NA,Yes,NA,Beer & Wine Only,Casual,Casual,NA,No,No,...,Yes,Yes,No,No,$11-30,Izakaya,4.0,230,10003,326 E 6th St
4,NA,NA,Yes,NA,Beer & Wine Only,Casual,Casual,NA,Yes,NA,...,No,Yes,NA,No,$11-30,Jintan,4.0,46,10009,10th St & 1st Ave
5,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,No,NA,NA,NA,NA,Suki,4.5,18,10003,86 E 7th St
6,No,NA,Yes,NA,Beer & Wine Only,Classy,Casual,NA,No,No,...,Yes,Yes,NA,No,Above $61,Kyo Ya,4.5,432,10009,94 E 7th St
7,NA,NA,Yes,NA,NA,NA,NA,NA,NA,NA,...,Yes,NA,NA,NA,NA,Restaurant Ukiyo,4.5,32,10003,239 E 5th St
8,No,NA,Yes,No,Beer & Wine Only,Casual,Casual,NA,Yes,No,...,No,NA,NA,No,$11-30,Soba-ya,4.0,1365,10003,229 E 9th St
9,No,NA,No,No,Beer & Wine Only,Casual,Casual,NA,Yes,Yes,...,Yes,Yes,No,No,$11-30,Ichibantei,4.0,182,10009,401 E 13th St


In [31]:
final_df.to_csv('sample.csv')

In [ ]:
print('Please enter a restaurant name below, if you have no idea about it, just click return')
name=input("restaurant name: ")

if name=='':
    #return the original csv
else:
    #return the selceted csv

In [ ]:
print('Please enter a restaurant style below, if you have no idea about it, just click return')
style=input("restaurant name: ")
if style=='':
    #return the last original csv
else:
    #return the selected csv

In [ ]:
print('Are you going to eat Lunch, Dinner or Breakfast?, please input Lunch or Dinner, if you have no idea about it, just click return')
mealkind=input("Lunch, Dinner or Breakfast :")
if mealkind=='':
    #return the last original csv
else:
    #return the selected csv

In [ ]:
print('Want to find a place to drink alcohol? Please enter Full Bar, Beer&Wine Only or No below')
alcohol=input("alcohol:")
if mealkind=='':
    #return the last original csv
else:
    #return the selected csv